In [1]:
#Install packages

!pip install python-twitter



In [50]:
# Import packages

import pandas as pd
import numpy as np
import twitter
import html
from nltk.stem import WordNetLemmatizer
#from nltk.stem.snowball import SnowballStemmer

In [2]:
api = twitter.Api(consumer_key = 'FP94BfUftKeygulmcLcVqRNvX',
                 consumer_secret = 'GfZtNuk6Zu6lTOYGWibXI95MjMyks6SSlBEykyLuYe4NEUgGUu',
                 access_token_key = '961088185756393472-FopLzpw7n3CrHhbHoWv8BlnR1mZwhGH',
                 access_token_secret= '62Trp0LVEPvAwoGPH4ov8D4TQe2eEaQETKIfXKCPZX6NN')


In [3]:
print(api.VerifyCredentials())

{"created_at": "Wed Feb 07 04:04:14 +0000 2018", "default_profile": true, "default_profile_image": true, "id": 961088185756393472, "lang": "en", "name": "IUSMM2018", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "iusmm2018"}


In [66]:
# create function to accept search term and tehn fetch the tweets for that term

def TestData(search_string):
    try:
        tweets_fetched = api.GetSearch(search_string, count=100)
        print("Fetched: "+str(len(tweets_fetched))+" tweets with the term "+search_string+"!!")
        return [{"text":status.text, "label":None} for status in tweets_fetched]        
    except:
        print("fetch_error: please check")
        return None
              

testData = TestData("%23HurricaneHarvey%20OR%20%23Harvey")
              

Fetched: 86 tweets with the term %23HurricaneHarvey%20OR%20%23Harvey!!


In [67]:
testData[0:9]

[{'label': None,
  'text': 'Born and raised in Houston and plays for @astrosbaseball Michael Bourn did his #MillionDollarPledge #10for10… https://t.co/mA9j1QKcde'},
 {'label': None,
  'text': "On Oct. 30, 2017, George P. Bush's agency signed a $13.47 million contract with Horne LLP for #Harvey disaster reco… https://t.co/c1EyOjTVbM"},
 {'label': None,
  'text': 'What would it take to set up 10,000 cots overnight? An army of #volunteers and an organization like @BakerRipley to… https://t.co/X6lfBjSCxI'},
 {'label': None,
  'text': 'Join 8 people right now at "Harvey Weinstein\'s lawyer says mogul\'s behavior had \'no criminality\'" #cheers… https://t.co/jVf0MSxQ2c'},
 {'label': None,
  'text': "#Harvey Weinstein's #Lawyer says ...  \nMore here : https://t.co/PVNOJHSkvG  'no #Criminality039 #Mogul039s… https://t.co/bnhmTH2Vcv"},
 {'label': None,
  'text': 'RT @DonnaWR8: .@POTUS #TRUMP🇺🇸 &amp; @FLOTUS🌺\n\nWhen ALL seemed HOPELESS...YOU brought HOPE!\n\nYou INSPIRE us ALL!\n\n#MAGA #Harvey

In [63]:
# Step 2:  To manually annotate the downloaded tweets based on relief terms in the tweets and/or write a function to create a corpus to use as training data.  

In [68]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords 

# Step 3:  Pre-process the Tweets

class PreProcessTweets:
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['ATUSER','URL','IMG'])
        
    def processTweets(self,list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"])))
        return processedTweets
    
    def _processTweet(self,tweet):
        # Unescape from HTML
        tweet = html.unescape(tweet)
        # 3a. Convert to lower case
        tweet = tweet.lower()
        # 3b. Replace links with the word URL 
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet) 
        # 3c. Replace @username with "AT_USER"
        tweet = re.sub('@[^\s]+','ATUSER',tweet)
        # 3d. Replace #word with word 
        tweet = re.sub(r'#([^\s]+)',r'\1',tweet)
        # 3e. Replace images with the word IMG 
        tweet = re.sub(r'\bpic.twitter.com\s+', 'IMG', tweet)
        # 3f Keep only words with letters
        tweet = re.sub('[^a-zA-Z]',' ',tweet)
        # 3g. Remove RT
        tweet = re.sub(r'\brt([\b\s])', '', tweet)
        # 3h. tokenize tweets
        tweet = word_tokenize(tweet)
        # Apply stemming
        # stemmer = SnowballStemmer("english")
        # Apply Lemmatization
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(word) for word in tweet if word not in self._stopwords]
    
tweetProcessor=PreProcessTweets()
#ppTrainingData=tweetProcessor.processTweets(trainingData)
ppTestData=tweetProcessor.processTweets(testData)

[nltk_data] Downloading package punkt to /Users/nescobar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nescobar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
print(ppTestData)

[['born', 'raised', 'houston', 'play', 'michael', 'bourn', 'milliondollarpledge'], ['oct', 'george', 'p', 'bush', 'agency', 'signed', 'million', 'contract', 'horne', 'llp', 'harvey', 'disaster', 'reco'], ['would', 'take', 'set', 'cot', 'overnight', 'army', 'volunteer', 'organization', 'like'], ['join', 'people', 'right', 'harvey', 'weinstein', 'lawyer', 'say', 'mogul', 'behavior', 'criminality', 'cheer'], ['harvey', 'weinstein', 'lawyer', 'say', 'criminality', 'mogul'], ['trump', 'seemed', 'hopeless', 'brought', 'hope', 'inspire', 'u', 'maga', 'harvey', 'usa'], ['many', 'sign', 'heaven', 'earth', 'coincidence', 'seek', 'truth', 'revelation', 'september', 'eclipse', 'harvey'], ['harvey', 'lingers', 'business', 'neighborhood', 'catmull'], ['mistake', 'molester', 'go', 'hide', 'away', 'somewhere', 'like', 'good', 'friend', 'harvey'], ['new', 'york', 'sue', 'weinstein', 'co', 'harvey', 'weinstein', 'sexu'], ['almost', 'month', 'since', 'hurricaneharvey', 'hit', 'devastated', 'houston', 'ti

In [ ]:
# Step 4: Extract features from both training and test data using multinomial Naive Bayes Classifier

#Step 4a : Build a Vocabulary

#Step 4b : Represent each tweet with presence/absence of the relief terms in the tweet

In [ ]:
# Step 5: Train the classifier on the training data

In [ ]:
# Step 6 : Use the classifier to classify the tweets

In [ ]:
# Step 7 : Model Inference & Visualization